In [0]:
# Google 드라이브를 마운트하려면 이 셀을 실행하세요.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
# 파일 불러오기

import os

def get_file_list(dir_name):
  return os.listdir(dir_name)

def get_contents(file_list):
  y_class = []
  X_text = []
  class_dict = {
      1: '0', 2:'0', 3:'0', 4:'0', 5: '1', 6:'1', 7:'1', 8:'1'}
  
  for file_name in file_list:
    try:
      f = open(file_name, 'r', encoding = 'cp949')
      category = int(file_name.split(os.sep)[6].split("_")[0])
      y_class.append(class_dict[category])
      X_text.append(f.read())
      f.close()
    except UnicodeDecodeError as e:
      print(e)
      print(file_name)
  return X_text, y_class

def get_cleaned_text(text):
  import re
  text = re.sub('\W+','', text.lower() )
  return text

def get_corpus_dict(text):
  text = [sentence.split() for sentence in text]
  cleaned_words = [get_cleaned_text(word) for words in text for word in words]
  
  from collections import OrderedDict
  corpus_dict = OrderedDict()
  for i, v in enumerate(set(cleaned_words)):
    corpus_dict[v] = i
  return corpus_dict

def get_count_vector(text, corpus):
  text = [sentence.split() for sentence in text]
  word_number_list = [[corpus[get_cleaned_text(word)]for word in words] for words in text]
  X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))] 

  for i, text in enumerate(word_number_list):
    for word_number in text:
      X_vector[i][word_number] += 1
      
  return X_vector

import math
def get_cosine_similarity(v1, v2):
  "compute cosine similarity of v1 to v2: (v1 dot v2) /{||v1||*||v2||}"
  sumxx, sumxy, sumyy = 0, 0, 0
  for i in range(len(v1)):
    x = v1[i]; y = v2[i]
    sumxx += x*x
    sumyy += y*y
    sumxy += x*y
  return sumxy/math.sqrt(sumxx*sumyy)

def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list

def get_top_n_similarity_news(similarity_score, n):
  import operator
  x = {i:v for i, v in enumerate(similarity_score)}
  sorted_x = sorted(x.items(), key=operator.itemgetter(1))
  
  return list(reversed(sorted_x))[1:n+1]

def get_accuracy(similarity_list, y_class, source_news):
  source_class = y_class[source_news]
  
  return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)

     
if __name__ == "__main__":
  dir_name = '/content/gdrive/My Drive/Python/news_data'
  file_list = get_file_list(dir_name)
  file_list = [os.path.join(dir_name, file_name) for file_name in file_list]
  
  X_text, y_class = get_contents(file_list)
  
  corpus = get_corpus_dict(X_text)
  print('Number of words : {0}'.format(len(corpus)))
  X_vector = get_count_vector(X_text, corpus)
  
  source_number = 10
  
  result = []
  
  for i in range(80):
    source_number = i
    
    similarity_score = get_similarity_score(X_vector, source_number)
    similarity_news = get_top_n_similarity_news(similarity_score, 10)
    accuracy_score = get_accuracy(similarity_news, y_class, source_number)
    result.append(accuracy_score)

  print(sum(result)/ 80)


Number of words : 4024
0.6950000000000001


In [0]:
!ls /content/gdrive

'My Drive'


In [22]:
# News Categorization using sklearn

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
corpus = [
    'This is the first document.',
    'This is the second second document',
    'And the third one',
    'Is this the first document?',
]

X = vectorizer.fit_transform(corpus)
X.toarray()

vectorizer.get_feature_names()


['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']